This file will learn follow reference https://towardsdatascience.com/named-entity-recognition-ner-meeting-industrys-requirement-by-applying-state-of-the-art-deep-698d2b3b4ede

In [1]:
def read_all_line( directory , file ):
    file = open( directory + "/" + file , "r" , encoding="utf-8" )
    contents = file.read()
    file.close()
    return contents

In [3]:
def individual_preprocess( raw_data , split_sentence , count ,remove = [] ):
    pre_data_frame = { "Sentence" : [] , "Word" : [] , "Tag" : [] }
    for word in raw_data:
        if word in remove :
            None
        elif word in split_sentence :
            count += 1
        else: 
            check = word.find('(')
            if check > 0:
                pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word[ 0 : word.find('(')] )
                pre_data_frame["Tag"].append( word[ word.find('(') + 1 : word.find(')')] )
            else:
                pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word )
                pre_data_frame["Tag"].append( "O" )
    return pre_data_frame , count

In [4]:
import pandas as pd
def preprocess( directory , list_file , count = 1 , individual = False ):
    data_frame = pd.DataFrame( { "Sentence" : [] , "Word" : [] , "Tag" : [] } )
    if( individual ):
        word = read_all_line( directory , list_file ).split('|')
        pre_data_frame , count = individual_preprocess( word ,
                                              ["\n"] ,
                                              count ,
                                              [ "" , '' , '\0'] )
        data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    else:
        for file in list_file :
            word = read_all_line( directory , file ).split('|')
            pre_data_frame , count = individual_preprocess( word ,
                                              ["\n"] ,
                                              count ,
                                              [ "" , '' , '\0'] )
            data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    return data_frame

In [5]:
#directory of corpus
corpus_directory = "test_in_class"
corpus_files = ("Politic9.txt" , "Allcolumn.txt" , "98JUL5_1.txt" ,
                "98JUL5_2.txt" , "bkknews1.txt" , "Result1.txt" ,
                "Result2(corpus-1).txt" , "Result3(corpus-2).txt" ,
                "result4.txt" , "Result_c_2_1.txt", "Result_c_2_2.txt" )
corpus_test = ( "POL1108.CUT" , "POL1805.CUT" , "POL1812.CUT" ,  
                "POL1819.CUT" , "POL2008.CUT" , "POL2017.CUT" ,
                "POL1109.CUT" , "POL1806.CUT" , "POL1813.CUT" , 
                "POL2001.CUT" , "POL2009.CUT" , "POL2018.CUT" ,
                "POL1111.CUT" , "POL1807.CUT" , "POL1814.CUT" , 
                "POL2002.CUT" , "POL2010.CUT" , "POL2019.CUT" ,
                "POL1112.CUT" , "POL1808.CUT" , "POL1815.CUT" ,
                "POL2004.CUT" , "POL2011.CUT" , "POL2020.CUT" ,
                "POL1113.CUT" , "POL1809.CUT" , "POL1816.CUT" ,
                "POL2005.CUT" , "POL2012.CUT" , "POL1803.CUT" ,
                "POL1810.CUT" , "POL1817.CUT" , "POL2006.CUT" ,
                "POL2014.CUT" , "POL1804.CUT" , "POL1811.CUT" ,
                "POL1818.CUT" , "POL2007.CUT" , "POL2015.CUT" )
all_corpus = corpus_files + corpus_test
individual = False

In [6]:
data_frame = preprocess( corpus_directory + "/train_data" , corpus_files )
data_frame
number_words = len( set( list( data_frame["Word"].values ) ) )
number_tags = len( list( set( data_frame["Tag"].values ) ) )
print( "Have type of tag is " + str( number_tags ) + 
        " tags from " + str(number_words) + " words")
# print(set( list( data_frame["Tag"].values )))

Have type of tag is 14 tags from 9293 words


In [7]:
data_frame

,Sentence,Word,Tag
0,Sentence: 1,"""",O
1,Sentence: 1,ทักษิณ,per
2,Sentence: 1,"""",O
3,Sentence: 1,,O
4,Sentence: 1,ขอ,O
...,...,...,...
103378,Sentence: 5825,ทะเล,org_end
103379,Sentence: 5825,,O
103380,Sentence: 5825,จังหวัด,O
103381,Sentence: 5825,ภูเก็ต,loc


Follow https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2

In [8]:
data_frame['Sentence'].nunique(), data_frame.Word.nunique(), data_frame.Tag.nunique()

(5825, 9293, 14)

In [9]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [10]:
data_frame = data_frame.fillna(method='ffill')
print( "Number of sentence is {:d}".format(data_frame['Sentence'].nunique()) )
print( "Number of word is {:d}".format( data_frame.Word.nunique() ) )
print( "Number of tag is {:d}".format( data_frame.Tag.nunique() ) )

Number of sentence is 5825
Number of word is 9293
Number of tag is 14


In [11]:
data_frame.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,,33
1,O,86629
2,loc,2989
3,loc_cont,91
4,loc_end,183
5,loc_start,183
6,org,3195
7,org_cont,2470
8,org_end,1529
9,org_start,1553


In [50]:
data_train = data_frame.drop('Tag', axis=1)

vector_handle = DictVectorizer(sparse=False)
data_train = vector_handle.fit_transform( data_train.to_dict('records') )

#value_train = data_frame.Tag.values

In [13]:
test_frame = preprocess( corpus_directory + "/test_tag" , corpus_test , 1 , individual )
#data_test = test_frame.drop('Tag', axis=1)
#vector_handle = DictVectorizer(sparse=False)
#data_test = vector_handle.fit_transform( data_test.to_dict('records') )
#value_test = test_frame.Tag.values

In [14]:
data_frame.shape[0] , test_frame.shape[0]

(103383, 16273)

In [48]:
all_frame = data_frame
all_data = all_frame.drop('Tag', axis=1)
vector_handle = DictVectorizer(sparse=False)
all_data = vector_handle.fit_transform(all_data.to_dict('records'))
all_value = all_frame.Tag.values
size_of_test = test_frame.shape[0]
# print( "Propotional of test is " + str( size_of_test ) )

classes = np.unique( all_value )
classes = classes.tolist()
new_classes = classes.copy()
new_classes.remove('O')
new_classes

['',
 'loc',
 'loc_cont',
 'loc_end',
 'loc_start',
 'org',
 'org_cont',
 'org_end',
 'org_start',
 'per',
 'per_cont',
 'per_end',
 'per_start']

In [51]:
test_data = test_frame.drop('Tag', axis=1)
vector_handle = DictVectorizer(sparse=False)
test_data = vector_handle.fit_transform(test_data.to_dict('records'))



In [17]:
X_train, X_test, y_train, y_test = train_test_split(all_data, 
        all_value, 
        test_size = size_of_test, 
        shuffle = False )

In [18]:
test_data.to_dict('records')

AttributeError: 'list' object has no attribute 'to_dict'

In [19]:
# Use Perceptron # the best from three model
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

TypeError: float() argument must be a string or a number, not 'dict'

In [30]:
X_train.shape , test_data.shape , X_test.shape

((87110, 15118), (16273, 2318), (16273, 15118))

In [29]:
per.predict( X_train[0] )

ValueError: Expected 2D array, got 1D array instead:
array=[1. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
print(classification_report(y_pred=per.predict(X_test), 
                            y_true=y_test, labels=new_classes))

In [ ]:
# Use Passive Aggressive Classifier # Not work
# pa =PassiveAggressiveClassifier()
# pa.partial_fit(X_train, y_train, classes)

In [ ]:
#print(classification_report(y_pred=pa.predict(X_test),
                            #y_true=y_test, labels=new_classes))

In [ ]:
# use naive bayes # NOT WORK
# nb = MultinomialNB(alpha=0.01)
# nb.partial_fit(X_train, y_train, classes)

In [ ]:
#print(classification_report(y_pred=nb.predict(X_test), 
                            #y_true=y_test, labels = new_classes))